In [1]:
from scipy.io import loadmat
import numpy as np
import graph_tool as gt
from graph_tool import inference
from pytictoc import TicToc
from joblib import Parallel, delayed
from readFile import *
from SBMs import *
import config

#set global variables from config
choose_run = config.choose_run
threshold = config.threshold
file_names = config.file_names
iterations = config.iterations
directory = config.loc

#set up function for parallelization
def obtain_nested_partition(i):
    #choose run with lowest percent censored frames and adjust index
    PC = int(choose_run['best_runs'][i]) - 1
    
    # load the data for that run
    print(file_names[i])
    adj = loadmat(directory + file_names[i])
    
    # select the correct connectivity matrix for best run
    rmat = np.asarray(adj['RS_connectivity_raw'][0][PC]['xcorr'][0][0]['rmat'])
    run = np.asarray(rmat[0][0])

    # obtain nSBM partition
    partition = np.asarray(nested_SBM(run, threshold, iterations))
    return partition

#initialize timing
t = TicToc()
t.tic()

#set range for desired indices within full dataset
start_index = 0
end_index = 3

#set number of available cores for parallelization
cores = 3

#parallelize according to number of available cores - verbose for job updates
results = Parallel(n_jobs=cores, verbose = 30, timeout = 30000)(delayed(obtain_nested_partition)(i) for i in range(start_index,end_index))

#end timing
t.toc()

#choose file name
file_name = "example_nested.csv"

#save results as csv - each row within csv file corresponds to a different subject and each column (of 1088) is the node of corresponding index
np.savetxt(file_name, results, delimiter=',')

Data directory:/home/daoutidi/shared/brain_transfer/
Current threshold: 0.59
Number of iterations set to 2


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   1 tasks      | elapsed:  1.0min


Elapsed time is 102.884961 seconds.


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.7min finished
